# tensorflow实现一个卷积层的前向传播过程
通过tf.get_variable的方式创建过滤器的权重和偏置项变量。上面介绍了卷集层的参数个数只和过滤器的尺寸、深度以及当前层节点的矩阵的深度有关，所以这里声明的参数变量是一个四维矩阵，前面两个代表了过滤器的尺寸，第三个维度表示当前层的深度，第四个维度表示过滤器的深度。

In [3]:
import tensorflow as tf
filter_weigths=tf.get_variable('weigths',[5,5,3,16],initializer=tf.truncated_normal_initializer(stddev=0.1))

和卷集层的权重类似，当前矩阵上不同位置的偏置项也是共享的，所以总共有下一层深度个数不同的偏置项。本样例代码中的16为过滤器的深度，也是神经网络中下一层节点矩阵的深度.

In [4]:
biases=tf.get_variable('biases',shape=[16],initializer=tf.constant_initializer(0.1))

tf.nn.conv2d提供了一个非常方便的函数来实现卷积层向前传播的算法。这个函数的第一个输入为当前层的节点矩阵。注意这个矩阵是一个四维矩阵，后面三个维度对应一个节点矩阵，第一维对应一个输入batch。比如在输入层，input[0,:,:,:]表示第一张图片，input[1,:,:,:]表示第二张图片，以此类推。tf.nn.conv2d第二个参数提供了卷集层的权重，第三个参数为不同维度上的步长，虽然第三个参数提供了一个长度为4的数组，但是第一维和最后一维的数字要求一定是1。这是因为卷积层的步长只对矩阵的长宽有效。最后一个参数是填充的方法，Tensorflow中提供SAME和VALID两种选择，其中SAME表示添加全0填充，VALID表示不添加。

In [ ]:
conv=tf.nn.conv2d(input,filter_weigths,strides=[1,1,1,1],padding='SAME')

tf.nn.bias_add提供了一个方便的函数给每个节点加上偏置项，注意这里不能直接使用加法，因为矩阵上不同位置上的节点都需要加上同样的偏置项。

In [ ]:
bias=tf.nn.bias_add(conv,biases)
#将计算结果通过relu函数完成去线性化
active_conv=tf.nn.relu(bias)

In [ ]:
#tf.nn.max_pool实现了最大池化层的前向传播过程，他的参数和tf.nn.conv2d函数类似
#ksize提供了过滤器的尺寸，strides提供了步长信息，padding提供了是否用全0填充。
pool=tf.nn.max_pool(actived_conv,ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME')

In [2]:
import tensorflow as tf
#调整输入数据placeholder的格式，输入为一个四维矩阵
x=tf.placeholder(tf.float32,[BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,NUM_CHANNELS],name='x_input')
#类似地将输入的格式调整为4维，并将这个调整后的数据传入sess.run过程
reshaped_xs=np.reshape(xs,(BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,NUM_CHANNELS))


NameError: name 'BATCH_SIZE' is not defined

In [1]:
import tensorflow as tf

#配置神经网络的参数
INPUT_NODE=784
OUTPUT_NODE=10

IMAGE_SIZE=28
NUM_CHANNELS=1
NUM_LABELS=10

#第一层卷积层的尺寸和深度
CONV1_DEEP=32
CONV1_SIZE=5

#第二层卷积层的尺寸和深度
CONV2_DEEP=64
CONV2_SIZE=5

#全连接层的节点个数
FC_SIZE=512

#定义卷积神经网络的前向传播过程，这里面添加了一个新的参数train,用于区分训练过程和测试过程这个过程中
#用到dropout方法，dropout可以进一步提升模型的可靠性并防止过拟合，dropout过程只在训练时使用

def inference(input_tensor,train,regularizer):
    #声明第一层的变量并实现前向传播过程
    #通过使用不同的命名空间来隔离不同层的变量
    #这样可以让每一层中的变量命名只需要考虑当前层的作用，而不需要担心重命名的问题
    #这里定义的卷积层输入为28*28*1的原始MNIST图片像素。
    #因为卷积层中使用了全0填充，所以输出为28*28*32的矩阵
    with tf.variable_scope('layer1-conv1'):
        conv1_weigths=tf.get_variable("weigths",[CONV1_SIZE,CONV1_SIZE,NUM_CHANNELS,CONV1_DEEP],initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv1_biases=tf.get_variable("bias",[CONV1_DEEP],initializer=tf.constant_initializer(0.0))
        #使用边长为5，深度为37的过滤器，过滤移动的步长为1，且使用全零补充
        conv1=tf.nn.conv2d(input_tensor,conv1_weigths,strides=[1,1,1,1],padding='SAME')
        relu1=tf.nn.relu(tf.nn.bias_add(conv1,conv1_biases))
        
        #实现第二层池化层的前向传播过程。这里选用最大池化层，池化层过滤器的边长为2，，使用全零补充并且移动的步长为2
        #这一层的输入是上一层的输出，也就是28*28*32的矩阵
        #输出为14*14*32的矩阵
    with tf.name_scope('layer2-pool1'):
        pool1=tf.nn.max_pool(relu1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
            
    #声明第三层的卷积层的变量并实现前向传播过程，这一层的输入为14*14*32的矩阵
    #输出为14*14*64的矩阵
    with tf.variable_scope('layer-conv2'):
        conv2_weigths=tf.get_variable("weigths",[CONV2_SIZE,CONV2_SIZE,CONV1_DEEP,CONV2_DEEP],initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv2_biases=tf.get_variable("bias",[CONV2_DEEP],initializer=tf.constant_initializer(0.0))
        #使用边长为5，深度为64的过滤器，过滤移动的步长为1，且使用全零补充。
        conv2=tf.nn.conv2d(pool1,conv2_weigths,strides=[1,1,1,1],padding='SAME')
        relu2=tf.nn.relu(tf.nn.bias_add(conv2,conv2_biases))
            
    #实现第四层池化层的前向传播过程，第一层和第二层的结构是一样的，这一层的输入为14*14*64的矩阵，输出为7*7*64的矩阵。
    with tf.name_scope('layer4_pool2'):
        pool2=tf.nn.max_pool(relu2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    
    #将第四层池化层的输出转化为第五层全连接层的输入格式，第四层的输出为7*7*64的矩阵，然而
    #第五层的全连接层需要的输入格式为向量，所以这里需要将这个7*7*64的矩阵拉成一个向量
    #pool2.get_shape()函数可以得到第四层输出矩阵的维度而不需要手工计算。
    #注意，因为每一层神经网络的输入输出都为一个batch的矩阵，所以这里得到的维度包含一个batch中数据的个数
    pool_shape=pool2.get_shape().as_list()
        
    #计算将矩阵拉直成向量之后的长度，这个长度就是矩阵长宽以及深度的乘积
    nodes=pool_shape[1]*pool_shape[2]*pool_shape[3]
    #通过tf.reshape函数将第四层的输出变成一个batch的向量
    reshaped=tf.reshape(pool2,[pool_shape[0],nodes])
        
    #第五层全连接层
    with tf.variable_scope('layer5-fcl'):
        fcl_weights=tf.get_variable("weight",[nodes,FC_SIZE],initializer=tf.truncated_normal_initializer(stddev=0.1))
        #只有全连接层的权重需要加入正则化
        if(regularizer!=None):
            tf.add_to_collection('losses',regularizer(fcl_weights))
        fcl_biases=tf.get_variable("bias",[FC_SIZE],initializer=tf.constant_initializer(0.1))
        fcl=tf.nn.relu(tf.matmul(reshaped,fcl_weights)+fcl_biases)
        if(train):
            fcl=tf.nn.dropout(fcl,0.5)
        #声明第六层变量并实现全连接层的前向传播结果。
    with tf.variable_scope('layer6-fc2'):
        fc2_weights=tf.get_variable("weight",[FC_SIZE,NUM_LABELS],initializer=tf.truncated_normal_initializer(stddev=0.1))
        if(regularizer!=None):
            tf.add_to_collection('losses',regularizer(fcl_weights))
        fc2_biases=tf.get_variable("bias",[NUM_LABELS],initializer=tf.constant_initializer(0.1))
        logit=tf.matmul(fcl,fc2_weights)+fc2_biases
    return logit
    


In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

#配置神经网络参数
BATCH_SIZE=100
LEARNING_RATE_BASE=0.8   #基础学习率
LEARNING_RATE_DECAY=0.99     #学习率的衰减率
REGULARAZTION_RATE=0.0001  #描述模型复杂度的正则化项在损失函数中的系数
TRAINING_STEPS=5000       #训练次数
MOVING_AVERAGE_DECAY=0.99     #滑动平均衰减率

#模型保存的路径和文件名
MODEL_SAVE_PATH="/path/to/model/"
MODEL_NAME="model.ckpt"


def train(mnist):
    # 定义输出为4维矩阵的placeholder
    x = tf.placeholder(tf.float32, [
            BATCH_SIZE,
            IMAGE_SIZE,
            IMAGE_SIZE,
            NUM_CHANNELS],
        name='x-input')
    y_ = tf.placeholder(tf.float32, [None,OUTPUT_NODE], name='y-input')
    
    #L2正则化损失函数
    regularizer=tf.contrib.layers.l2_regularizer(REGULARAZTION_RATE)
    #直接使用前向传播算法
    y=inference(x,False,regularizer)
    #定义训练轮次的变量
    global_step=tf.Variable(0,trainable=False)
    
    #接下来定义损失函数，学习率，滑动平均值，以及训练过程
    variable_averages=tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
    variable_averages_op=variable_averages.apply(tf.trainable_variables())
    
    #交叉熵
    cross_entropy=tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,labels=tf.argmax(y_,1))
    cross_entropy_mean=tf.reduce_mean(cross_entropy)
    
    #总损失
    loss=cross_entropy_mean+tf.add_n(tf.get_collection('losses'))
    
    #设置学习率和优化损失函数
    learning_rate=tf.train.exponential_decay(LEARNING_RATE_BASE,global_step,mnist.train.num_examples/BATCH_SIZE,LEARNING_RATE_DECAY)
    train_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step)
    with tf.control_dependencies([train_step,variable_averages_op]):
        train_op=tf.no_op(name='train')
        
    #初始化持久化类
    saver=tf.train.Saver()
    with tf.Session() as sess:
        tf.initialize_all_variables().run()
        for i in range(TRAINING_STEPS):
            xs,ys=mnist.train.next_batch(BATCH_SIZE)
            reshaped_xs = np.reshape(xs, (
                BATCH_SIZE,
                IMAGE_SIZE,
                IMAGE_SIZE,
                NUM_CHANNELS))
            _,loss_value,step=sess.run([train_op,loss,global_step],feed_dict={x:reshaped_xs,y_:ys})
            
            #每1000轮保存一次模型
            if(i%1000==0):
                print("After %d training step(s),loss on training batch is %g." %(step,loss_value))
                saver.save(sess,os.path.join(MODEL_SAVE_PATH,MODEL_NAME),global_step=global_step)

def main(argv=None):
    mnist=input_data.read_data_sets("/path/to/MNIST_data/",one_hot=True)
    train(mnist)
if __name__=='__main__':
    tf.app.run()

Extracting /path/to/MNIST_data/train-images-idx3-ubyte.gz
Extracting /path/to/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /path/to/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /path/to/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.
After 1 training step(s),loss on training batch is 6.60094.


# 卷积神经网络手写数字识别系统最佳实践

In [1]:
#构建LeNet-5模型结构的前向传播过程
import tensorflow as tf
import numpy as np
#配置神经网络的参数
INPUT_NODE=784
OUTPUT_NODE=10

IMAGE_SIZE=28
NUM_CHANNELS=1
NUM_LABELS=10

#第一层卷积层的尺寸和深度
CONV1_DEEP=32
CONV1_SIZE=5
#第二层的尺寸和深度
CONV2_DEEP=64
CONV2_SIZE=5
#全连接层的节点个数
FC_SIZE=512

def inference(input_tensor,train,regularizer):
    with tf.variable_scope('layer1-conv1'):
        conv1_weights=tf.get_variable("weights",[CONV1_SIZE,CONV1_SIZE,NUM_CHANNELS,CONV1_DEEP],initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv1_biases=tf.get_variable("bias",[CONV1_DEEP],initializer=tf.constant_initializer(0.0))
        conv1=tf.nn.conv2d(input_tensor,conv1_weights,strides=[1,1,1,1],padding='SAME')
        relu1=tf.nn.relu(tf.nn.bias_add(conv1,conv1_biases))
    with tf.variable_scope('layer2-pool1'):
        pool1=tf.nn.max_pool(relu1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    with tf.variable_scope('layer3-conv2'):
        conv2_weights=tf.get_variable("weights",[CONV2_SIZE,CONV2_SIZE,CONV1_DEEP,CONV2_DEEP],initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv2_biases=tf.get_variable("bias",[CONV2_DEEP],initializer=tf.constant_initializer(0.0))
        conv2=tf.nn.conv2d(pool1,conv2_weights,strides=[1,1,1,1],padding='SAME')
        relu2=tf.nn.relu(tf.nn.bias_add(conv2,conv2_biases))
    with tf.variable_scope('layer4-pool2'):
        pool2=tf.nn.max_pool(relu2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
   
    pool1_shape=pool2.get_shape().as_list()
    nodes=pool1_shape[1]*pool1_shape[2]*pool1_shape[3]
    reshape=tf.reshape(pool2,[pool1_shape[0],nodes])
    
    with tf.variable_scope('layer5-fc1'):
        fc1_weights=tf.get_variable("weights",[nodes,FC_SIZE],initializer=tf.truncated_normal_initializer(stddev=0.1))
        #只有全连接层才需要加入正则化
        if(regularizer!=None):
            tf.add_to_collection('losses',regularizer(fc1_weights))
        fc1_biases=tf.get_variable("bias",[FC_SIZE],initializer=tf.constant_initializer(0.1))
        fc1=tf.nn.relu(tf.matmul(reshape,fc1_weights)+fc1_biases)
        if(train):
            fc1=tf.nn.dropout(fc1,0.5)
    with tf.variable_scope('layer5-fc2'):
        fc2_weights=tf.get_variable("weights",[FC_SIZE,NUM_LABELS],initializer=tf.truncated_normal_initializer(stddev=0.1))
        if(regularizer!=None):
            tf.add_to_collection('losses',regularizer(fc2_weights))
        fc2_biases=tf.get_variable("bias",[NUM_LABELS],initializer=tf.constant_initializer(0.1))
        logit=tf.matmul(fc1,fc2_weights)+fc2_biases
    return logit

In [2]:
#构建LeNet-5模型结构的训练过程
import os
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

#配置神经网络的参数
BATCH_SIZE=100
LEARNING_RATE_BASE=0.8    #基础的学习率
LEARNING_RATE_DECAY=0.99     #学习率的衰减率
REGULARAZTION_RATE=0.0001    #描述模型复杂度的正则化项在损失函数中的系数
TRAINING_STEP=5000           #训练轮次
MOVING_AVERAGE_DECAY=0.99     #滑动平均衰减率
#模型保存的路径和文件名
MODEL_SAVE_PATH="/path/Tensorflow/6/"
MODEL_NAME="model.ckpt"
def train(mnist):
    #定义输入输出placeholder
    x=tf.placeholder(tf.float32,[BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,NUM_CHANNELS],name='X_input')
    y_=tf.placeholder(tf.float32,[None,OUTPUT_NODE],name='y_input')
    regularizer=tf.contrib.layers.l2_regularizer(REGULARAZTION_RATE)
    y=inference(x,False,regularizer)
    global_step=tf.Variable(0,trainable=False)
    
    #定义损失函数、学习率、滑动平均操作以及训练过程
    variable_averages=tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
    variable_averages_op=variable_averages.apply(tf.trainable_variables())
    cross_entropy=tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,labels=tf.argmax(y_,1))
    cross_entropy_mean=tf.reduce_mean(cross_entropy)
    loss=cross_entropy_mean+tf.add_n(tf.get_collection('losses'))
    learning_rate=tf.train.exponential_decay(LEARNING_RATE_BASE,global_step,mnist.train.num_examples/BATCH_SIZE,LEARNING_RATE_DECAY,staircase=True)
    train_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
    with tf.control_dependencies([train_step,variable_averages_op]):
        train_op=tf.no_op(name='train')
    
    #初始化Tensorflow持久化类
    saver=tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        for i in range(TRAINING_STEP):
            xs,ys=mnist.train.next_batch(BATCH_SIZE)
            reshaped_xs=np.reshape(xs,(BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,NUM_CHANNELS))
            _,loss_value,step=sess.run([train_op,loss,global_step],feed_dict={x:reshaped_xs,y_:ys})
            
            if(i%1000==0):
                print("After %d training step(s),loss on training batch is %g." %(step,loss_value))
                saver.save(sess,os.path.join(MODEL_SAVE_PATH,MODEL_NAME),global_step=global_step)

In [3]:
def main(argv=None):
    mnist=input_data.read_data_sets("/path/to/MNIST_data/",one_hot=True)
    train(mnist)
if __name__=='__main__':
    tf.app.run()

Extracting /path/to/MNIST_data/train-images-idx3-ubyte.gz
Extracting /path/to/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /path/to/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /path/to/MNIST_data/t10k-labels-idx1-ubyte.gz
After 1 training step(s),loss on training batch is 4.2587.
After 1001 training step(s),loss on training batch is 17.4877.
After 2001 training step(s),loss on training batch is 15.2794.
After 3001 training step(s),loss on training batch is 13.4302.
After 4001 training step(s),loss on training batch is 11.867.


SystemExit: 

C:\Users\pujing\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [9]:
import time
import tensorflow as tf
EVAL_INTERVAL_SECS=10
def evaluate(mnist):
    with tf.Graph().as_default() as g:
        #定义输入输出placeholder
        x=tf.placeholder(tf.float32,[mnist.validation.num_examples,IMAGE_SIZE,IMAGE_SIZE,NUM_CHANNELS],name='X_input')
        y_=tf.placeholder(tf.float32,[None,OUTPUT_NODE],name='y_input')
        
        xs,ys=mnist.validation.next_batch(mnist.validation.num_examples)
        
        reshaped_xs=np.reshape(xs,(mnist.validation.num_examples,IMAGE_SIZE,IMAGE_SIZE,NUM_CHANNELS))
        validate_feed={x:reshaped_xs,y_:mnist.validation.labels}
        
        y=inference(x,False,None)
        correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
        accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        variable_averages=tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
        variables_to_restore=variable_averages.variables_to_restore()
        saver=tf.train.Saver(variables_to_restore)
        while True:
            with tf.Session() as sess:
                ckpt=tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
                if(ckpt and ckpt.model_checkpoint_path):
                    #加载模型
                    saver.restore(sess,ckpt.model_checkpoint_path)
                    #通过文件名保存模型保存时迭代的轮次
                    global_step=ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                    accuracy_score=sess.run(accuracy,feed_dict=validate_feed)
                    print("After %s training step(s),validation accuracy=%g " % (global_step,accuracy_score))
                else:
                    print("No ckickpoint file found")
                    return
                time.sleep(EVAL_INTERVAL_SECS)
        
        

In [10]:
def main(argv=None):
    mnist=input_data.read_data_sets("/path/to/MNIST_data/",one_hot=True)
    evaluate(mnist)
if __name__=='__main__':
    main()

Extracting /path/to/MNIST_data/train-images-idx3-ubyte.gz
Extracting /path/to/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /path/to/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /path/to/MNIST_data/t10k-labels-idx1-ubyte.gz


ValueError: Shape must be rank 4 but is rank 5 for 'layer1-conv1/Conv2D' (op: 'Conv2D') with input shapes: [?,5000,28,28,1], [5,5,1,32].

In [1]:
import tensorflow as tf
INPUT_NODE = 784
OUTPUT_NODE = 10

IMAGE_SIZE = 28
NUM_CHANNELS = 1
NUM_LABELS = 10

CONV1_DEEP = 32
CONV1_SIZE = 5

CONV2_DEEP = 64
CONV2_SIZE = 5

FC_SIZE = 512

def inference(input_tensor, train, regularizer):
    with tf.variable_scope('layer1-conv1'):
        conv1_weights = tf.get_variable(
            "weight", [CONV1_SIZE, CONV1_SIZE, NUM_CHANNELS, CONV1_DEEP],
            initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv1_biases = tf.get_variable("bias", [CONV1_DEEP], initializer=tf.constant_initializer(0.0))
        conv1 = tf.nn.conv2d(input_tensor, conv1_weights, strides=[1, 1, 1, 1], padding='SAME')
        relu1 = tf.nn.relu(tf.nn.bias_add(conv1, conv1_biases))

    with tf.name_scope("layer2-pool1"):
        pool1 = tf.nn.max_pool(relu1, ksize = [1,2,2,1],strides=[1,2,2,1],padding="SAME")

    with tf.variable_scope("layer3-conv2"):
        conv2_weights = tf.get_variable(
            "weight", [CONV2_SIZE, CONV2_SIZE, CONV1_DEEP, CONV2_DEEP],
            initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv2_biases = tf.get_variable("bias", [CONV2_DEEP], initializer=tf.constant_initializer(0.0))
        conv2 = tf.nn.conv2d(pool1, conv2_weights, strides=[1, 1, 1, 1], padding='SAME')
        relu2 = tf.nn.relu(tf.nn.bias_add(conv2, conv2_biases))

    with tf.name_scope("layer4-pool2"):
        pool2 = tf.nn.max_pool(relu2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        pool_shape = pool2.get_shape().as_list()
        nodes = pool_shape[1] * pool_shape[2] * pool_shape[3]
        reshaped = tf.reshape(pool2, [pool_shape[0], nodes])

    with tf.variable_scope('layer5-fc1'):
        fc1_weights = tf.get_variable("weight", [nodes, FC_SIZE],
                                      initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer != None: tf.add_to_collection('losses', regularizer(fc1_weights))
        fc1_biases = tf.get_variable("bias", [FC_SIZE], initializer=tf.constant_initializer(0.1))

        fc1 = tf.nn.relu(tf.matmul(reshaped, fc1_weights) + fc1_biases)
        if train: fc1 = tf.nn.dropout(fc1, 0.5)

    with tf.variable_scope('layer6-fc2'):
        fc2_weights = tf.get_variable("weight", [FC_SIZE, NUM_LABELS],
                                      initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer != None: tf.add_to_collection('losses', regularizer(fc2_weights))
        fc2_biases = tf.get_variable("bias", [NUM_LABELS], initializer=tf.constant_initializer(0.1))
        logit = tf.matmul(fc1, fc2_weights) + fc2_biases

    return logit

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os
import numpy as np

BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.01
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 6000
MOVING_AVERAGE_DECAY = 0.99

MODEL_SAVE_PATH="/path/Tensorflow/7/"
MODEL_NAME="model.ckpt"

def train(mnist):
    # 定义输出为4维矩阵的placeholder
    x = tf.placeholder(tf.float32, [
            BATCH_SIZE,
            IMAGE_SIZE,
            IMAGE_SIZE,
            NUM_CHANNELS],
        name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')
    
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    y =inference(x,False,regularizer)
    global_step = tf.Variable(0, trainable=False)

    # 定义损失函数、学习率、滑动平均操作以及训练过程。
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples / BATCH_SIZE, LEARNING_RATE_DECAY,
        staircase=True)

    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name='train')
        
    # 初始化TensorFlow持久化类。
    saver = tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        for i in range(TRAINING_STEPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)

            reshaped_xs = np.reshape(xs, (
                BATCH_SIZE,
                IMAGE_SIZE,
                IMAGE_SIZE,
                NUM_CHANNELS))
            _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x: reshaped_xs, y_: ys})

            if i % 1000 == 0:
                print("After %d training step(s), loss on training batch is %g." % (step, loss_value))
                saver.save(sess,os.path.join(MODEL_SAVE_PATH,MODEL_NAME),global_step=global_step)

In [ ]:
def main(argv=None):
    mnist=input_data.read_data_sets("/path/to/MNIST_data/",one_hot=True)
    train(mnist)
if __name__=='__main__':
    tf.app.run()

Extracting /path/to/MNIST_data/train-images-idx3-ubyte.gz
Extracting /path/to/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /path/to/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /path/to/MNIST_data/t10k-labels-idx1-ubyte.gz
After 1 training step(s), loss on training batch is 4.3068.
After 1001 training step(s), loss on training batch is 0.681441.
After 2001 training step(s), loss on training batch is 0.736585.
After 3001 training step(s), loss on training batch is 0.699708.
After 4001 training step(s), loss on training batch is 0.657589.
After 5001 training step(s), loss on training batch is 0.709574.


In [ ]:
import time
import tensorflow as tf
EVAL_INTERVAL_SECS=10
def evaluate(mnist):
    with tf.Graph().as_default() as g:
        #定义输入输出placeholder
        x=tf.placeholder(tf.float32,[mnist.validation.num_examples,IMAGE_SIZE,IMAGE_SIZE,NUM_CHANNELS],name='X_input')
        y_=tf.placeholder(tf.float32,[None,OUTPUT_NODE],name='y_input')
        
        xs,ys=mnist.validation.next_batch(mnist.validation.num_examples)
        
        reshaped_xs=np.reshape(xs,(mnist.validation.num_examples,IMAGE_SIZE,IMAGE_SIZE,NUM_CHANNELS))
        validate_feed={x:reshaped_xs,y_:mnist.validation.labels}
        
        y=inference(x,False,None)
        correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
        accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        variable_averages=tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
        variables_to_restore=variable_averages.variables_to_restore()
        saver=tf.train.Saver(variables_to_restore)
        while True:
            with tf.Session() as sess:
                ckpt=tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
                if(ckpt and ckpt.model_checkpoint_path):
                    #加载模型
                    saver.restore(sess,ckpt.model_checkpoint_path)
                    #通过文件名保存模型保存时迭代的轮次
                    global_step=ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                    accuracy_score=sess.run(accuracy,feed_dict=validate_feed)
                    print("After %s training step(s),validation accuracy=%g " % (global_step,accuracy_score))
                else:
                    print("No ckickpoint file found")
                    return
                time.sleep(EVAL_INTERVAL_SECS)

In [ ]:
def main(argv=None):
    mnist=input_data.read_data_sets("/path/to/MNIST_data/",one_hot=True)
    evaluate(mnist)
if __name__=='__main__':
    main()

## Tensorflow实现迁移学习

In [2]:
import glob
import os.path
import random
import numpy as np
import tensorflow as tf
from tensorflow.python.platform import gfile

In [ ]:
#Inception-v3模型瓶颈层的节点个数
BOTTLENECK_TENSOR_SIZE=2048

#Inception-v3模型中代表瓶颈层结构的张量名称
# 在谷歌提出的Inception-v3模型中，这个张量名称就是'pool_3/_reshape:0'。
# 在训练模型时，可以通过tensor.name来获取张量的名称。
BOTTLENECK_TENSOR_NAME='pool_3/_reshape:0'

#图像输入张量对应的名称
JPEG_DATA_TENSOR_NAME='DecodeJpeg/contents:0'

#下载的谷歌训练好的Inception-v3模型文件目录
MODEL_DIR='model/'

#下载的谷歌训练好的Inception-v3模型文件名
MODEL_FILE='tensorflow_inception_graph.pb'

#因为一个训练数据可能会被使用多次，所以可以将原图像通过Inception-v3模型计算得到的特征向量保存在文
#件中，免去重复计算
#下面定义了这些文件的存放地址。
CACHE_DIR='tmp/bottleneck/'


#图片数据文件夹
#在这个文件夹中每个子文件夹代表一个需要区分的类别，每个子文件夹中存放了对应类别的图片
INPUT_DATA='flower_data'

#验证的数据百分比
VALIDATION_PERCENTAGE=10

#测试数据的百分比
TEST_PERCENTAGE=10

#定义神经网络的设置

LEARNING_RATE=0.01
STEPS=4000
BATCH=100

#这个函数从数据文件夹中读取所有的图片列表并按照训练验证测试数据分开
#testing_percentage和yalidation_perdation_percentage参数指定了测试数据集合验证数据集的大小
def create_image_lists(testing_percentage,validation_perecentage):
    #得到的所有图片都存在result这个字典里
    #这个字典的key为类别的名称，value也是一个字典，字典里存储了所有图片名称。
    result={}
    #获取当前名录下所有的子目录
    sub_dirs=[x[0] for x in os.walk(INPUT_DATA)]
    
    #得到的第一个目录是当前目录不需要考虑
    is_root_dir=True
    for sub_dir in sub_dirs:
        if(is_root_dir):
            is_root_dir=False
            continue
        #获取当前目录下所有的有效图片文件
        extensions=['jpg','jpeg','JPG','JPEG']
        file_list=[]
        dir_name=os.path.basename(sub_dir)
        for extension in extensions:
            file_glob=os.path.join(INPUT_DATA,dir_name,'*.'+extension)
            file_list.extend(glob.glob(file_glob))
        if not file_list:
            continue
            
        
            